<a href="https://colab.research.google.com/github/kake01/Bot/blob/master/%E5%AD%A6%E7%BF%92%E3%83%99%E3%83%BC%E3%82%B9%E8%BF%94%E7%AD%94%E3%81%BE%E3%81%A7%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境構築

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [47]:
from google.colab import drive
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
import MeCab
import random
import pandas as pd
wakati = MeCab.Tagger("-Owakati")

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


# Dock2Vecに対応したデータへの前処理 & 返答の辞書作成

In [17]:
sentences = []
advise = []
dictionary = {}
send_ms = '朝食は抜きでした。昼食は、ラーメンとご飯を食べました。午前、午後共に、車での移動なので運動は特になし。夕食はスープとハヤシライスでした。運動は家に帰ってからの犬の散歩と、階段の上り下りです。明け方まで起きていたので、夜食に焼きそばを食べました。'
sentence_num = 0
with open('drive/My Drive/Health/AfterPreprocessingData.tsv', 'r', encoding='utf-8') as f:
  lines = f.readlines()
  for line in lines:
    # 配列から##と\n文字の削除
    line = line.replace('##', '')
    line = line.replace('\n', '')
    
    # 同じ文書だった場合、辞書にレスポンスを追加
    if send_ms == line[:line.find('|')]:
      dictionary.setdefault(sentence_num, []).append(line[line.find('|')+1:])
    else:
      sentences.append(send_ms)
      sentence_num += 1
    send_ms = line[:line.find('|')]

In [21]:
# print(sentences[0])
# print(sentences[1])
# print(sentences[2])
# print(dictionary[1])
print(len(sentences))

542


# ファイルの読み込み & sentenceの作成

In [ ]:
sentences = []
with open('drive/My Drive/Health/Dock2VecDataSet.tsv', 'r', encoding='utf-8') as f:
  lines = f.readlines()
  for line in lines:
    line = line.replace('\n', '')
    # 分かち書きを行う
    test = wakati.parse(line).split()
    sentences.append(test)

In [24]:
wakati_sentences = []
for i in sentences:
  test = wakati.parse(i).split()
  wakati_sentences.append(test)

In [25]:
print(sentences[0])
print(sentences[1])
print(wakati_sentences[0])

朝食は抜きでした。昼食は、ラーメンとご飯を食べました。午前、午後共に、車での移動なので運動は特になし。夕食はスープとハヤシライスでした。運動は家に帰ってからの犬の散歩と、階段の上り下りです。明け方まで起きていたので、夜食に焼きそばを食べました。
朝食はオープンサンドとコーヒーでした。昼食は焼きそばとふかしいもでした。午後に掃除機をかけました。夕方にウオーキングと運動をして、そのあとスーパーに買い物に行きました。夕食は八宝菜、茶碗蒸し、味噌汁、玄米でした。赤ワインを飲みました。
['朝食', 'は', '抜き', 'でし', 'た', '。', '昼食', 'は', '、', 'ラーメン', 'と', 'ご飯', 'を', '食べ', 'まし', 'た', '。', '午前', '、', '午後', '共に', '、', '車', 'で', 'の', '移動', 'な', 'ので', '運動', 'は', '特に', 'なし', '。', '夕食', 'は', 'スープ', 'と', 'ハヤシライス', 'でし', 'た', '。', '運動', 'は', '家', 'に', '帰っ', 'て', 'から', 'の', '犬', 'の', '散歩', 'と', '、', '階段', 'の', '上り', '下り', 'です', '。', '明け方', 'まで', '起き', 'て', 'い', 'た', 'ので', '、', '夜食', 'に', '焼きそば', 'を', '食べ', 'まし', 'た', '。']


# Dock2Vecを使用してモデルの作成 & 保存

In [26]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(wakati_sentences)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, epochs=10, workers=4)

# 新規文章の類似度計測と返答の作成

## 井熊のやり方

In [ ]:
x = model.infer_vector(wakati.parse("どうも井熊です。").split())
print(model.docvecs.most_similar([x]))

## 先生のやり方

In [73]:
def ResponseText(text):
  max_value = 0
  max_num = 0
  text_wakati = wakati.parse(text).split()
  for i in range(len(wakati_sentences)):
    sim_value = model.docvecs.similarity_unseen_docs(model, text_wakati, wakati_sentences[i], alpha=1, min_alpha=0.0001, steps=5)
    if max_value < sim_value:
      max_value = sim_value
      max_num = i

    rand = random.randint(0,len(dictionary[i]))
  return dictionary[max_num][rand]
  #類似度Max
  #return sentences[i]

"""
while True:
  text = input(">>>")
  if text == "終了":
    break
"""

anser = ResponseText("今日は起床が遅く、朝ごはんが抜きで昼はカップ麺でした。夜は自炊をしハンバーガーを食べた。野菜はほとんど食べなかった。運動はしていない。")
print(anser)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


冬の健康料理はなんといっても鍋です。


In [43]:
def DebugResponseText(text):
  max_value = 0
  max_num = 0
  text_wakati = wakati.parse(text).split()
  for i in range(len(wakati_sentences)):
    sim_value = model.docvecs.similarity_unseen_docs(model, text_wakati, wakati_sentences[i], alpha=1, min_alpha=0.0001, steps=5)
    if max_value < sim_value:
      max_value = sim_value
      max_num = i
    #max_value = max(max_value, sim_value)
  return max_value, max_num, wakati_sentences[max_num],sentences[max_num]
  #類似度Max
  #return sentences[i]

"""
while True:
  text = input(">>>")
  if text == "終了":
    break
"""

anser = DebugResponseText("今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。")
print(anser)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


(0.9852368, 448, ['朝', 'は', 'パン', 'を', '食べ', 'まし', 'た', '。', '昼', 'は', '日', '替り', 'の', 'お', '弁当', 'を', '食べ', 'まし', 'た', '。', '夜', 'は', 'パン', 'と', '肉まん', 'を', '食べ', 'まし', 'た', '。', '運動', 'は', '通勤', 'を', '兼ね', 'て', '往復', '一', '時間', '歩い', 'て', 'い', 'ます', '。'], '朝はパンを食べました。昼は日替りのお弁当を食べました。夜はパンと肉まんを食べました。運動は通勤を兼ねて往復一時間歩いています。')


# 実験のゴミ

## most_similarを使用

In [29]:
def process_query(query):
  text_wakati = wakati.parse(query).split()
  return text_wakati

query = "今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。"
l = process_query(query)
sim = model.most_similar(positive=l,topn=10)
print(sim)

[('カレー', 0.9996607899665833), ('焼きそば', 0.9994827508926392), ('お昼', 0.9994096159934998), ('夕食', 0.9993243217468262), ('ラーメン', 0.9991921186447144), ('おでん', 0.9991458654403687), ('うどん', 0.9991227388381958), ('パスタ', 0.9989135265350342), ('カレーライス', 0.9986866116523743), ('肉まん', 0.9986369609832764)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## 実際にモデルを使用して類似度検索を行う

In [ ]:
# 朝食と昼食の類似度
print(model.docvecs.most_similar(0))
print(model.docvecs.similarity(0,0))

In [34]:
def SameSentence(text):
    text_wakati = wakati.parse(text).split()
    print(text_wakati)
    # documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(text_wakati)]
    # print(documents)
    # similarity = model.similarity(documents,0)
    # print(model.docvecs.most_similar(["myfizzbuzz.py"]))
    # test = model.most_similar(documents)
    test = model.most_similar(text_wakati)
    return test

print(SameSentence("今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。"))

['今日', 'は', '起床', 'が', '遅く', '、', '朝食', 'を', 'とる', '時間', 'が', '無く', '食べ', 'ませ', 'ん', 'でし', 'た', '。', '昼', 'は', 'コンビニ', 'で', '買っ', 'た', 'サンドイッチ', 'を', '食べ', 'まし', 'た', '。', '夜', 'は', '野菜', 'を', '多め', 'に', '摂り', 'まし', 'た', '。']
[('カレー', 0.9996607899665833), ('焼きそば', 0.9994827508926392), ('お昼', 0.9994096159934998), ('夕食', 0.9993243217468262), ('ラーメン', 0.9991921186447144), ('おでん', 0.9991458654403687), ('うどん', 0.9991227388381958), ('パスタ', 0.9989135265350342), ('カレーライス', 0.9986866116523743), ('肉まん', 0.9986369609832764)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
